# Final exam

**For this exam, feel free to re-use any code from the previous lab notebooks.**

#### Tasks
- Use accelerator data to construct a neural network surrogate model, train that model, and demonstrate that it accurately models the data
- Use Bayesian optimization to optimize the function and determine the best operational parameters

## Setting up the environment

In [2]:
%reset -f

import numpy as np
import matplotlib.pyplot as plt

#add PyTorch and TorchVision (used for cropping etc.)
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
import torch
import scipy.constants as cst

def propagate(bunch_n_particles=1000, bunch_rms_z=1.e-3,
        bunch_mean_E=100e6, bunch_rms_E=0.01e6,
        linac_final_E=1000e6, linac_rf_frequency=1.3e9, linac_phase=0.,
        arc_r56=0, arc_r566=0 ):
        """
        Compute the propagation of an electron bunch through the linac + arc

        Parameters:
        -----------
        bunch_n_particles: int
            Number of particles in the bunch

        bunch_rms_z: float (in meters)
            The RMS size of the bunch along z (before entering the linac)

        bunch_mean_E: float (in eV)
            The mean energy of the bunch (before entering the linac)

        bunch_rms_E: float (in eV)
            The RMS energy spread of the bunch (before entering the linac)

        linac_final_E: float (in eV)
            The (mean) energy of the bunch at the end of the linac

        linac_rf_frequency: float (in Hz)
            The frequency of the RF cavity of the linac

        linac_phase: float (in degrees)
            The phase of the bunch in the linac

        arc_r56, arc_r566: floats (in meter)
            The coeficients of the energy-dependent delay induced by the arc:
            z -> z + r56*delta + t566*delta**2

        Returns
        -------
        rms_z : float (meters)
            Longitudinal bunch length

        rms_delta : float (None)
            RMS bunch energy deviation from reference

        """
        # Generate the bunch before the linac, with random Gaussian distribution
        bunch_z = torch.randn(bunch_n_particles) * bunch_rms_z
        bunch_delta = torch.randn(bunch_n_particles) * bunch_rms_E / bunch_mean_E

        # Analytical change in relative energy spread (delta) after the bunch propagates in the linac
        # $\delta \rightarrow \delta \frac{E_0}{E_1} + (1- \frac{E_0}{E_1})\frac{\cos(kz +\phi)}{\cos(\phi)}$
        k = 2*cst.pi*linac_rf_frequency/cst.c
        phi = linac_phase * 2*cst.pi/360. # Convert from degrees to radians
        E0_over_E1 = bunch_mean_E/linac_final_E
        bunch_delta = E0_over_E1 * bunch_delta.clone() + \
          (1. - E0_over_E1)*(torch.cos(k*bunch_z + phi)/torch.cos(phi) -1)

        # Analytical change in position (z) after the bunch propagates in the arc
        # z -> z + r56*delta + t566*delta**2
        bunch_z = bunch_z + arc_r56*bunch_delta + \
                        arc_r566*bunch_delta**2

        #add noise to the observations
        bunch_delta += torch.randn(1)*1e-4
        bunch_z += torch.randn(1)*1e-3

        return torch.hstack((bunch_z.std(), bunch_delta.std())).reshape(1,-1)

## Generate dataset

The code below generates samples from "simulations" of a beam propagating through a linac followed by an arc:
- The input to the simulator are the linac's phase, the arc's R56 coefficient, and the beam final (mean) energy.
- The output of the simulator are the bunch length (in meters) and the bunch's RMS energy spread.

In [4]:
#generate input samples
n = 10000
torch.manual_seed(0)
inputs = torch.rand((n, 3))

#phase +/- 50 deg
inputs[:, 0] = inputs[:, 0] * 100.0 - 50.0

#r56 +/- 0.5
inputs[:,1] = inputs[:,1] - 0.5

#final energy [800,1300]*1e6
inputs[:,2] = (inputs[:,2] * 500.0 + 800.0) * 1e6

print('Inputs:')
print(inputs)

outputs = []
for i in range(n):
    outputs += [propagate(linac_phase = inputs[i][0],
                                        arc_r56 = inputs[i][1],
                                        linac_final_E = inputs[i][2])]

outputs = torch.vstack(outputs)

print('Outputs:')
print(outputs)

Inputs:
tensor([[-3.7434e-01,  2.6822e-01,  8.4424e+08],
        [-3.6797e+01, -1.9258e-01,  1.1170e+09],
        [-9.9066e-01,  3.9644e-01,  1.0278e+09],
        ...,
        [ 2.6369e+01, -4.5239e-01,  8.7906e+08],
        [-1.9793e+01, -1.4001e-02,  1.1869e+09],
        [-4.6165e+00, -1.8174e-01,  8.3491e+08]])
Outputs:
tensor([[0.0010, 0.0005],
        [0.0026, 0.0189],
        [0.0011, 0.0006],
        ...,
        [0.0063, 0.0118],
        [0.0009, 0.0089],
        [0.0006, 0.0021]])


## Regression with a neural network

**Task:**
    Appropriately normalize the input data, and standardize the output data.

**Task:**
    Create and train a neural network to model this data, i.e. for each data point, the neural network should take the above 3 (normalized) input features and predict the above 2 (normalized) output features.
    
In order to show that the neural network works as expected and that the training roughly converged, plot the evolution of the loss function - both for the training dataset and test dataset - during training. (Use the first 7000 data points as the training set, and the remaining 3000 data points as the test set.)
</div>

## Bayesian optimization

**Task:**
    Use Bayesian optimization to minimize the bunch length (i.e. the first of the 2 features that are returned by `propagate`) with respect to phase, r56, and final_energy (i.e. the 3 input features that we passed to `propagate` when previously generating the dataset for the neural network). DO NOT USE XOPT.
    
Use 3 randomly generated points in the input domain (shown in the comments above) as the initial dataset on which to fit the initial Gaussian Process (at the beginning of Bayesian optimization). Then, at each iteration of Bayesian optimization, call the `propagate` function on the new candidate point.
    
Run 6 steps of Bayesian optimization and print the values of the bunch length obtained at each iteration. What is the best value obtained so far?